# Tomato Disease Diagnosis

In [ ]:
from fastai.vision.all import *
import params
from fastcore.all import *
import wandb

In [ ]:
Path.BASE_PATH = path = Path("/notebooks/tomatodiagnosis/data/")

In [ ]:
path.ls()

Here we define several functions to help us process the data and upload it as a Table to W&B.

In [ ]:
def label_func(fname):
    return (fname.parent.name)

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split", "Label", "Dataset"])
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        label = label_func(image_file) 
        table.add_data(
            str(image_file.parent.name) + '/' + image_file.name, 
            wandb.Image(image),
            "None",
            label,
            "Tomato"
        )    
    return table

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

Let's add the images

In [ ]:
raw_data_at.add_dir(path/'Tomato_Yellow_Leaf_Curl_Virus', name='Tomato_Yellow_Leaf_Curl_Virus')
raw_data_at.add_dir(path/'healthy', name='healthy')
raw_data_at.add_dir(path/'Late_blight', name='Late_blight')
raw_data_at.add_dir(path/'Septoria_leaf_spot', name='Septoria_leaf_spot')
raw_data_at.add_dir(path/'Leaf_Mold', name='Leaf_Mold')
raw_data_at.add_dir(path/'Spider_mites', name='Spider_mites')
raw_data_at.add_dir(path/'Tomato_mosaic_virus', name='Tomato_mosaic_virus')
raw_data_at.add_dir(path/'Early_blight', name='Early_blight')
raw_data_at.add_dir(path/'Bacterial_spot', name='Bacterial_spot')
raw_data_at.add_dir(path/'Target_Spot', name='Target_Spot')

Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.

In [ ]:
image_files = get_image_files(path)

In [ ]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our run.

In [ ]:
raw_data_at.add(table, "eda_table3")

In [ ]:
run.log_artifact(raw_data_at)
run.finish()